# **AP155 Lab Assignment**
## Module 5: Partial Differential Equations

_Instructions_: Answer each problem as completely as you can. Discuss **all** your answers as clearly and concisely as possible.

_Scoring Criteria_: 50% - *correctness of code*; 50% - *discussion of the code and results*. Maximum score is **100 points**.



### Student Information

_Full Name (Last Name, First Name)_: Uyangoren, Seanuel\
_Student No._: 2021-07208\
_Section_: FX-3

### Submission Information

_Date and Time Submitted (most recent upload)_:

**HONOR PLEDGE** I affirm that I have upheld the highest principles of honesty and integrity in my academic work and that this lab assignment is my own work.

**Sign here with your full name:** Seanuel Joash Uyangoren

### Grading Information (c/o Lab Instructor)

TOTAL SCORE: **[]**/100

Score breakdown:
* Problem 1 - []/100

_Date and Time Scored (MM/DD/YYYY HH:MM AM/PM):_

### PROBLEM 1
**The Schrodinger equation and the Crank-Nicolson method**

_Refer to Exercise 9.8 in the Newman text._ In this problem, you will use the Crank-Nicolson method to solve the full time-dependent Schrodinger equation and hence develop a picture of how a wavefunction evolves over time.

Consider an electron (mass $M = 9.109 \times 10^{-31}$ kg) in a box of length $L = 10^{-8}$ m. Suppose that at time $t = 0$ the wavefunction of the electron has the form

$$ \psi(x,0) = \exp\left[-\frac{(x-x_0)^2}{2\sigma^2}\right]e^{i\kappa x},$$
where $x_0 = \frac{L}{2}$, $\sigma = 1 \times 10^{-10}$ m, $\kappa = 5 \times 10^{10} {\rm m}^{-1}$,  and $\psi = 0$ on the walls at $x = 0$ and $x = L$.

1. Perform a single step of the Crank-Nicolson method for this electron, calculating the vector $\psi(t)$ of values of the wavefunction, given the initial wavefunction above and using $N = 1000$ spatial slices with $a = L/N$. Your program will have to perform the following steps. First, given the vector $\psi(0)$ at $t = 0$, you will have to multiply by the matrix $\bf{B}$ to get a vector $\bf{v} = \bf{B}\psi$. Because of the tridiagonal form of $\bf{B}$, this is fairly simple. The $i$th component of $\bf{v}$ is given by
$$ v_i = b_1\psi_i + b_2(\psi_{i+1} + \psi_{i-1}).$$

   You will also have to choose a value for the time-step $h$. A reasonable choice is $h = 10^{-18}$ s. *(30 pts.)*

2. Second you will have to solve the linear system ${\bf Ax}= {\bf v}$ for $\bf{x}$, which gives you the new value of $\psi$. You could do this using a standard linear equation solver like the function $\tt solve$ in numpy's $\tt linalg$. *(20 pts.)*

3. Once you have the code in place to perform a single step of the calculation, extend your program to perform repeated steps and hence solve for $\psi$ at a sequence of times a separation $h$ apart. Note that the matrix $\bf A$ is independent of time, so it doesn't change from one step to another. You can set up the matrix just once and then keep on reusing it for every step. *(30 pts.)*

4. Make an animation of the solution by displaying the real part of the wavefunction at each time-step. You can use the function rate from the package visual to ensure a smooth frame-rate for your animation-- see Section 3.5 on page 117 of the Newman text.

   Run your animation for a while and describe what you see. Write a few sentences explaining in physics terms what is going on in the system. *(20 pts.)*

## Function to solve for x from Ax = v

In [3]:
from numpy import copy

def banded(Aa,va,up,down):
  A = copy(Aa)
  v = copy(va)
  N = len(v)

  # Gaussian elimination
  for m in range(N):

    # Normalization factor
    div = A[up,m]

    # Update the vector first
    v[m] /= div
    for k in range(1,down+1):
      if m+k<N:
        v[m+k] -= A[up+k,m]*v[m]

    # Now normalize and subtract the pivot row
    for i in range(up):
      j = m + up - i
      if j<N:
        A[i,j] /= div
        for k in range(1,down+1):
          A[i+k,j] -= A[up+k,m]*A[i,j]

  # Backsubstitution
  for m in range(N-2,-1,-1):
    for i in range(up):
      j = m + up - i
      if j<N:
        v[m] -= A[i,j]*v[j]

  return v

# Main Code

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#define initial wavefunction
def psi0(x, x0, sigma, kappa):
  return np.exp(-((x-x0)**2) / (2*sigma**2))*np.exp(1j*kappa*x)


#define constants
m = 9.109e-31
L = 1.0e-8
N = 1000
a = L/N
h = 1.0e-18

h_bar = 1.0545718e-34

x0 = L/2
sigma = 1.0e-10
kappa = 5.0e+10

#define equations to determine the elements in the tridiagonal array
a1 = 1 + h*(1j*h_bar)/(2*m*a**2)
a2 = - h*(1j*h_bar)/(4*m*a**2)
b1 = 1 - h*(1j*h_bar)/(2*m*a**2)
b2 = h*(1j*h_bar)/(4*m*a**2)

#define array where the wavefunctions will be stored
psi = np.zeros(N+1, complex)

#define array containing values for the x-axis
x_values = np.linspace(0, L, N + 1)

psi[:] = psi0(x_values, x0, sigma, kappa)     #call the initial wavefunction, psi0, then assign its values to psi
psi[[0,N]] = 0     #boundary condition 

#create a matrix A with a2 in the first row, a1 in the second, and a2 in the third row
#this will be used to create the tridiagonal later, through the banded function
A = np.empty((3,N), complex)
A[0,:] = a2     #first row
A[1,:] = a1     #second row
A[2:,] = a2     #third row



#Crank-Nicolson method
for i in range(5500):
  v = b1*psi[1:N] + b2*(psi[2:N+1] + psi[0:N-1])    #create an array v
  psi[1:N] = banded(A,v,1,1)        #send the array A to the banded function. In the function, we solve for x from Ax = v.
  
  #Generate and save the wavefunction plot every 50th time-step
  if i % 50 == 0:  
   plt.plot(x_values, psi)
   plt.ylim(-1,1)
   plt.xlabel("x")
   plt.ylabel("ψ")
   plt.savefig(str(i))
   plt.show()





NameError: name 'banded' is not defined

## Animation

### See animation here:

https://drive.google.com/file/d/1XeY0GGgTy5ZWPza5LDv0zIzLiKOf6weB/view?usp=sharing

We can see in the animation how the wave packet is located around x=L/2 initially, then starts moving to the right. This movement is caused by the initial momentum component introduced by the exp(ikx) term. At the boundaries, x = 0 and x = L, the wavefunction is zero. Hence, the wave packet reflects. 

After reflection, we see how the amplitude of the wave packet decreases. This is because when a reflected wave encounters an incident wave, and they are out-of-phase with each other, then destructive interference occurs.

We can also see how the wave packet spreads out over time. This is because of the gaussian term in the wavefunction. By the uncertainty principle, since the uncertainty of the position increases (as shown by the wave packet being spread out along x),  the uncertainty of momentum decreases. 

